<a href="https://colab.research.google.com/github/JimKing100/nfl-test/blob/master/predictions-final/Defense_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Installs
%%capture
!pip install pmdarima

In [2]:
# Imports
import numpy as np
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
from sklearn import preprocessing

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Import data
original_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-actuals/actuals_defense.csv')
player_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/players.csv')
bye_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/bye.csv')

In [4]:
# The dataframe of actual defense points for each game from 2000-2019
original_df.head()

,player,first,last,name,position1,2000-game1,2000-game2,2000-game3,2000-game4,2000-game5,2000-game6,2000-game7,2000-game8,2000-game9,2000-game10,2000-game11,2000-game12,2000-game13,2000-game14,2000-game15,2000-game16,2001-game1,2001-game2,2001-game3,2001-game4,2001-game5,2001-game6,2001-game7,2001-game8,2001-game9,2001-game10,2001-game11,2001-game12,2001-game13,2001-game14,2001-game15,2001-game16,2002-game1,2002-game2,2002-game3,...,2017-game9,2017-game10,2017-game11,2017-game12,2017-game13,2017-game14,2017-game15,2017-game16,2018-game1,2018-game2,2018-game3,2018-game4,2018-game5,2018-game6,2018-game7,2018-game8,2018-game9,2018-game10,2018-game11,2018-game12,2018-game13,2018-game14,2018-game15,2018-game16,2019-game1,2019-game2,2019-game3,2019-game4,2019-game5,2019-game6,2019-game7,2019-game8,2019-game9,2019-game10,2019-game11,2019-game12,2019-game13,2019-game14,2019-game15,2019-game16
0,SF,San Francisco,49ers,San Francisco 49ers,DF,-1.0,1.0,2.0,9.0,2.0,4.0,1.0,2.0,3.0,3.0,8.0,16.0,19.0,4.0,8.0,1.0,5.0,6.0,5.0,8.0,3.0,15.0,6.0,3.0,0.0,17.0,14.0,8.0,20.0,9.0,3.0,17.0,12,6,12,...,4,5,2,3,6,3,12,10,7.0,4.0,-1.0,8.0,0.0,2.0,-1.0,9.0,11.0,1.0,1.0,0.0,3.0,3.0,8.0,-5.0,24,7,6,14,9,9,16,3,19,14,10,3,-3,1,7,1
1,CHI,Chicago,Bears,Chicago Bears,DF,2.0,-1.0,9.0,7.0,8.0,9.0,4.0,8.0,12.0,3.0,19.0,3.0,0.0,3.0,8.0,15.0,5.0,10.0,26.0,14.0,8.0,14.0,11.0,5.0,10.0,9.0,10.0,5.0,13.0,11.0,26.0,15.0,7,11,6,...,3,5,4,5,10,6,11,8,14.0,17.0,13.0,13.0,5.0,2.0,4.0,27.0,12.0,14.0,13.0,4.0,17.0,8.0,6.0,7.0,8,5,21,14,4,-2,4,4,7,5,4,4,2,1,1,8
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,3.0,9.0,-1.0,5.0,12.0,2.0,8.0,8.0,1.0,2.0,4.0,3.0,8.0,1.0,4.0,11.0,5.0,23.0,1.0,7.0,5.0,2.0,10.0,2.0,5.0,9.0,9.0,13.0,1.0,8.0,20.0,7.0,0,1,0,...,6,10,5,3,1,3,7,9,12.0,12.0,0.0,0.0,22.0,-1.0,1.0,19.0,-5.0,4.0,-3.0,4.0,2.0,10.0,2.0,10.0,6,-1,5,2,1,9,2,0,-2,8,6,8,5,1,1,12
3,BUF,Buffalo,Bills,Buffalo Bills,DF,9.0,8.0,2.0,3.0,3.0,10.0,1.0,15.0,10.0,20.0,12.0,6.0,3.0,-1.0,4.0,13.0,2.0,10.0,2.0,-3.0,11.0,1.0,11.0,13.0,0.0,3.0,1.0,10.0,12.0,10.0,7.0,2.0,2,14,3,...,-3,-4,7,5,4,9,7,4,-1.0,1.0,14.0,6.0,11.0,15.0,-3.0,2.0,0.0,10.0,7.0,5.0,2.0,3.0,7.0,13.0,7,6,11,3,10,11,4,7,4,9,10,11,3,17,2,7
4,DEN,Denver,Broncos,Denver Broncos,DF,11.0,13.0,18.0,13.0,10.0,13.0,22.0,3.0,6.0,18.0,8.0,12.0,9.0,14.0,10.0,13.0,5.0,10.0,2.0,16.0,9.0,7.0,18.0,-1.0,12.0,10.0,6.0,2.0,12.0,11.0,10.0,2.0,8,7,12,...,-2,4,4,11,11,4,4,12,12.0,3.0,6.0,1.0,4.0,7.0,31.0,3.0,4.0,7.0,11.0,13.0,7.0,7.0,1.0,6.0,0,1,0,7,11,17,0,7,2,7,3,7,15,5,5,5


In [5]:
# The dataframe of all 2019 players
player_df.head()

,player,first,last,name,position1,start
0,SF,San Francisco,49ers,San Francisco 49ers,DF,2000
1,CHI,Chicago,Bears,Chicago Bears,DF,2000
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,2000
3,BUF,Buffalo,Bills,Buffalo Bills,DF,2000
4,DEN,Denver,Broncos,Denver Broncos,DF,2000


In [6]:
# The dataframe of bye weeks for 2019 teams
bye_df.head()

,player,first,last,name,position1,bye-week
0,SF,San Francisco,49ers,San Francisco 49ers,DF,4
1,CHI,Chicago,Bears,Chicago Bears,DF,6
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,9
3,BUF,Buffalo,Bills,Buffalo Bills,DF,6
4,DEN,Denver,Broncos,Denver Broncos,DF,10


In [0]:
# Add a row to the final_df dataframe
# Each row represents the predicted points for each team
def add_row(df, p, f, l, n, pos, wp, wa, pred, act):

    df = df.append({'player': p,
                    'first': f,
                    'last': l,
                    'name': n,
                    'position': pos,
                    wp: pred,
                    wa: act
                    }, ignore_index=True)
    
    return df

In [0]:
# The main code for iterating through the player(defense) list, calculating the points and adding the rows
# to the final_df dataframe.
def main(w):
    week_pname = 'week' + str(w) + '-pred'
    week_aname = 'week' + str(w) + '-act'

    column_names = ['player',
                    'first',
                    'last',
                    'name',
                    'position', 
                    week_pname,
                    week_aname
                     ]

    defense_df = player_df.loc[player_df['position1'] == 'DF']
    player_list = defense_df['player'].tolist()
    row = 0
    col = 308 + w

    week_df = pd.DataFrame(columns = column_names)

    for player in player_list:

        first = player_df['first'].loc[(player_df['player']==player)].iloc[0]
        last = player_df['last'].loc[(player_df['player']==player)].iloc[0]
        name = player_df['name'].loc[(player_df['player']==player)].iloc[0]
        position1 = player_df['position1'].loc[(player_df['player']==player)].iloc[0]

        bye_week = bye_df['bye-week'].loc[(bye_df['player']==player)].iloc[0]

        if w > bye_week:
            new_col = col - 1
        else:
            new_col = col

        if player == 'HOU':
            train_data = original_df.iloc[row, 37:new_col]
        else:
            train_data = original_df.iloc[row, 5:new_col]
        actuals = original_df.iloc[row, new_col:325]
        act_points = actuals.sum()
        row = row + 1

        print(player)
        print(bye_week)
        print(new_col)

        # ARIMA model
        model = pm.auto_arima(train_data, start_p=1, start_q=1,
                          test='adf',       # use adftest to find optimal 'd'
                          max_p=3, max_q=3, # maximum p and q
                          m=1,              # frequency of series
                          d=None,           # let model determine 'd'
                          seasonal=False,   # No Seasonality
                          start_P=0, 
                          D=0, 
                          trace=True,
                          error_action='ignore',  
                          suppress_warnings=True, 
                          stepwise=True)
    
        # Forecast
        if w > bye_week:
            n_periods = 17 - w + 1
        else:
            n_periods = 17 - w
        print(n_periods)
        fc = model.predict(n_periods=n_periods, return_conf_int=False)
        index_of_fc = np.arange(len(train_data), len(train_data)+n_periods)
        fc_series = pd.Series(fc, index=index_of_fc)
        pred_points = fc_series.sum()

        week_df = add_row(week_df, player, first, last, name, position1, week_pname, week_aname, pred_points, act_points)
    return week_df

In [9]:
week = 17
final_df = main(week)

SF
4
324
Performing stepwise search to minimize aic
Fit ARIMA: (1, 0, 1)x(0, 0, 0, 0) (constant=True); AIC=2027.891, BIC=2042.952, Time=0.636 seconds
Fit ARIMA: (0, 0, 0)x(0, 0, 0, 0) (constant=True); AIC=2029.581, BIC=2037.111, Time=0.017 seconds
Fit ARIMA: (1, 0, 0)x(0, 0, 0, 0) (constant=True); AIC=2031.570, BIC=2042.865, Time=0.074 seconds
Fit ARIMA: (0, 0, 1)x(0, 0, 0, 0) (constant=True); AIC=2031.571, BIC=2042.866, Time=0.070 seconds
Fit ARIMA: (0, 0, 0)x(0, 0, 0, 0) (constant=False); AIC=2294.899, BIC=2298.664, Time=0.011 seconds
Fit ARIMA: (2, 0, 1)x(0, 0, 0, 0) (constant=True); AIC=2029.081, BIC=2047.907, Time=0.695 seconds
Fit ARIMA: (1, 0, 2)x(0, 0, 0, 0) (constant=True); AIC=2028.813, BIC=2047.639, Time=0.753 seconds
Fit ARIMA: (0, 0, 2)x(0, 0, 0, 0) (constant=True); AIC=2032.864, BIC=2047.925, Time=0.129 seconds
Fit ARIMA: (2, 0, 0)x(0, 0, 0, 0) (constant=True); AIC=2032.867, BIC=2047.927, Time=0.169 seconds
Fit ARIMA: (2, 0, 2)x(0, 0, 0, 0) (constant=True); AIC=2036.779, 

In [10]:
final_df.head(50)

,player,first,last,name,position,week17-pred,week17-act
0,SF,San Francisco,49ers,San Francisco 49ers,DF,6.581570,1
1,CHI,Chicago,Bears,Chicago Bears,DF,5.900792,8
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,6.670846,12
3,BUF,Buffalo,Bills,Buffalo Bills,DF,6.978056,7
4,DEN,Denver,Broncos,Denver Broncos,DF,7.002121,5
5,CLE,Cleveland,Browns,Cleveland Browns,DF,6.164577,2
6,TB,Tampa Bay,Buccaneers,Tampa Bay Buccaneers,DF,7.148369,15
7,ARI,Arizona,Cardinals,Arizona Cardinals,DF,7.056426,0
8,LAC,Los Angeles,Chargers,Los Angeles Chargers,DF,6.893417,1
9,KC,Kansas City,Chiefs,Kansas City Chiefs,DF,6.990596,7


In [0]:
week_pred = 'week' + str(week) + '-pred'
week_act = 'week' + str(week) + '-act'
week_diff = 'week' + str(week) + '-diff'
week_pct = 'week' + str(week) + '-pct'

In [0]:
# The final_df dataframe
final_df[week_diff] = final_df[week_pred] - final_df[week_act]
final_df[week_pct] = final_df[week_diff] / final_df[week_pred]
final_df[week_pred] = final_df[week_pred].astype(int)

In [13]:
# Calculate the metrics
pred_median_error = final_df[week_pct].median()

print('Median Error - %.4f%%' % (pred_median_error * 100))

Median Error - 19.8848%


In [14]:
final_df.head(50)

,player,first,last,name,position,week17-pred,week17-act,week17-diff,week17-pct
0,SF,San Francisco,49ers,San Francisco 49ers,DF,6,1,5.58157,0.848061
1,CHI,Chicago,Bears,Chicago Bears,DF,5,8,-2.09921,-0.35575
2,CIN,Cincinnati,Bengals,Cincinnati Bengals,DF,6,12,-5.32915,-0.798872
3,BUF,Buffalo,Bills,Buffalo Bills,DF,6,7,-0.0219436,-0.00314465
4,DEN,Denver,Broncos,Denver Broncos,DF,7,5,2.00212,0.285931
5,CLE,Cleveland,Browns,Cleveland Browns,DF,6,2,4.16458,0.675566
6,TB,Tampa Bay,Buccaneers,Tampa Bay Buccaneers,DF,7,15,-7.85163,-1.09838
7,ARI,Arizona,Cardinals,Arizona Cardinals,DF,7,0,7.05643,1
8,LAC,Los Angeles,Chargers,Los Angeles Chargers,DF,6,1,5.89342,0.854934
9,KC,Kansas City,Chiefs,Kansas City Chiefs,DF,6,7,-0.00940439,-0.00134529
